In [2]:
%pip install --upgrade --quiet transformers
%pip install accelerate


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 25.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "NousResearch/Hermes-2-Pro-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [2]:
print(tokenizer.chat_template["tool_use"])

{%- macro json_to_python_type(json_spec) %}
{%- set basic_type_map = {
    "string": "str",
    "number": "float",
    "integer": "int",
    "boolean": "bool"
} %}

{%- if basic_type_map[json_spec.type] is defined %}
    {{- basic_type_map[json_spec.type] }}
{%- elif json_spec.type == "array" %}
    {{- "list[" +  json_to_python_type(json_spec|items) + "]"}}
{%- elif json_spec.type == "object" %}
    {%- if json_spec.additionalProperties is defined %}
        {{- "dict[str, " + json_to_python_type(json_spec.additionalProperties) + ']'}}
    {%- else %}
        {{- "dict" }}
    {%- endif %}
{%- elif json_spec.type is iterable %}
    {{- "Union[" }}
    {%- for t in json_spec.type %}
      {{- json_to_python_type({"type": t}) }}
      {%- if not loop.last %}
        {{- "," }} 
    {%- endif %}
    {%- endfor %}
    {{- "]" }}
{%- else %}
    {{- "Any" }}
{%- endif %}
{%- endmacro %}


{{- bos_token }}
{{- "You are a function calling AI model. You are provided with function signatures w

In [3]:
import requests
import random
from datetime import datetime
import pytz
import time
import json

def get_weather_forecast(location: str) -> dict[str, str]:
    """
    Retrieves a simple weather forecast for a given location.

    Args:
        location: The name of the location to get the weather forecast for.

    Returns:
        dict[str, str]: A dictionary containing the location, forecast, and temperature.
            If successful, the keys are 'location', 'forecast', and 'temperature'.
            If unsuccessful, the key is 'error' with a corresponding error message.

    Raises:
        requests.RequestException: If there's an error in making the API request.
    """
    url = f"https://wttr.in/{location}?format=%C,%t"
    response = requests.get(url)
    if response.status_code == 200:
        condition, temperature = response.text.strip().split(',')
        return {
            "location": location,
            "forecast": condition,
            "temperature": temperature
        }
    else:
        return {"error": "Unable to fetch weather data"}

def get_stock_price(symbol: str) -> str:
    """
    Retrieves the stock price for a given symbol.

    Args:
        symbol: The stock symbol to look up.

    Returns:
        str: The current stock price or an error message.
    """
    api_key = "your_stock_api_key"  # Replace with your actual API key
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        
        if "Global Quote" in data and "05. price" in data["Global Quote"]:
            return f"${data['Global Quote']['05. price']}"
        elif "Note" in data:
            return f"API limit reached: {data['Note']}"
        else:
            return f"Unable to fetch stock price for {symbol}. Response: {data}"
    
    except requests.RequestException as e:
        return f"Error fetching stock price: {str(e)}"
    except (KeyError, ValueError) as e:
        return f"Error parsing stock price data: {str(e)}"

def get_random_number(min_value: int, max_value: int) -> int:
    """
    Returns a random number between min_value and max_value (inclusive).

    Args:
        min_value: The minimum value of the range.
        max_value: The maximum value of the range.

    Returns:
        int: A random integer between min_value and max_value (inclusive).
    """
    return random.randint(min_value, max_value)

def get_current_time(time_zone: str, format: str) -> str:
    """
    Returns the current time in the specified time zone and format.

    Args:
        time_zone: The name of the time zone (e.g., 'America/New_York').
        format: The desired output format (e.g., '%Y-%m-%d %H:%M:%S').

    Returns:
        str: The current time formatted according to the specified format.

    Raises:
        pytz.exceptions.UnknownTimeZoneError: If the specified time zone is invalid.
    """
    tz = pytz.timezone(time_zone)
    current_time = datetime.now(tz)
    return current_time.strftime(format)

def speak_to_user(assistant_message: str) -> str:
    """
    Opens a text input widget for the user to provide feedback or confirm something.

    Args:
        assistant_message: The message to display to the user.

    Returns:
        str: The user's input as a string.
    """
    print(assistant_message)
    user_input = input("Please provide your feedback or confirmation: ")
    time.sleep(5)  # Wait for 5 seconds
    return user_input

def get_user_location(accuracy: int) -> str:
    """
    Returns the user's location based on the public IP address and accuracy level.
    
    Args:
        accuracy: The level of detail for the location information.
            1 - Country only
            2 - City and country
            3 - City, region, and country
    
    Returns:
        str: The location information based on the specified accuracy level or an error message.

    Raises:
        requests.RequestException: If there's an error in making the API requests.
        KeyError: If the response doesn't contain the expected data.
    """
    try:
        # Retrieve public IP address
        ip_response = requests.get('https://api.ipify.org?format=json')
        ip_response.raise_for_status()
        ip_address = ip_response.json().get('ip')
        
        # Use public IP to get location data
        location_url = f"http://ip-api.com/json/{ip_address}"
        location_response = requests.get(location_url)
        location_response.raise_for_status()
        data = location_response.json()
        
        if data['status'] == 'fail':
            return f"Error in get_user_location: {data.get('message', 'Unknown error')}"
        
        if accuracy == 1:
            return data.get("country", "Unknown country")
        elif accuracy == 2:
            return f"{data.get('city', 'Unknown city')}, {data.get('country', 'Unknown country')}"
        elif accuracy == 3:
            return f"{data.get('city', 'Unknown city')}, {data.get('regionName', 'Unknown region')}, {data.get('country', 'Unknown country')}"
        else:
            return "Invalid accuracy level. Please specify 1 (Country), 2 (City and Country), or 3 (City, Region, and Country)."
    except requests.RequestException as e:
        return f"Error: {e}"

In [5]:
def test_functions():
    print("Testing get_weather_forecast:")
    try:
        weather = get_weather_forecast("London")
        print(f"Weather in London: {weather}")
    except Exception as e:
        print(f"Error in get_weather_forecast: {str(e)}")

    print("\nTesting get_stock_price:")
    try:
        price = get_stock_price("AAPL")
        print(f"Current price of AAPL: ${price:.2f}")
    except Exception as e:
        print(f"Error in get_stock_price: {str(e)}")

    print("\nTesting get_random_number:")
    try:
        number = get_random_number(2, 42)
        print(f"Random number between 1 and 100: {number}")
    except Exception as e:
        print(f"Error in get_random_number: {str(e)}")

    print("\nTesting get_current_time:")
    try:
        time = get_current_time("America/New_York", "%Y-%m-%d %H:%M:%S")
        print(f"Current time in New York: {time}")
    except Exception as e:
        print(f"Error in get_current_time: {str(e)}")

    print("\nTesting get_user_location:")
    try:
        location = get_user_location(2)
        print(f"Location for user's ip address: {location}")
    except Exception as e:
        print(f"Error in get_user_location: {str(e)}")

In [7]:
test_functions()

Testing get_weather_forecast:


Weather in London: {'location': 'London', 'forecast': 'Sunny', 'temperature': '+67°F'}

Testing get_stock_price:
Error in get_stock_price: Unknown format code 'f' for object of type 'str'

Testing get_random_number:
Random number between 1 and 100: 25

Testing get_current_time:
Current time in New York: 2024-08-13 07:01:56

Testing get_user_location:
Location for user's ip address: Ashburn, United States


In [4]:
tools = [
    get_weather_forecast,
    get_stock_price,
    get_random_number,
    get_current_time,
    get_user_location,
    speak_to_user
]

In [44]:
messages = [
    {"role": "user", "content": "Get the current weather forecast for San Francisco and get stock price of Tesla"}
]

In [ ]:
messages = [
    {"role": "user", "content": "Get the user's location first. Once you have the correct user's location, get current weather forecast. Call the functions one at a time sequentially without commenting or asking for confirmation"}
]

In [32]:
prompt = tokenizer.apply_chat_template(
    messages,
    tools=tools,
    tokenize=False
)
print(prompt)

<|begin_of_text|>You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools> {"type": "function", "function": {"name": "get_weather_forecast", "description": "get_weather_forecast(location: str) -> dict[str, str] - Retrieves a simple weather forecast for a given location.

    Args:
        location(str): The name of the location to get the weather forecast for.
    Returns:
        dict[str, str]: A dictionary containing the location, forecast, and temperature.
        If successful, the keys are 'location', 'forecast', and 'temperature'.
        If unsuccessful, the key is 'error' with a corresponding error message.", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The name of the location to get the weather forecast for

In [33]:
def run_hermes_tool_inference(messages, tokenizer, model, tools):
    inputs = tokenizer.apply_chat_template(messages, chat_template="tool_use", tools=tools, add_generation_prompt=True, return_dict=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    out = model.generate(**inputs, max_new_tokens=512)
    return tokenizer.decode(out[0][len(inputs["input_ids"][0]):])

In [34]:
response = run_hermes_tool_inference(messages, tokenizer, model, tools)
print(response)

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


<tool_call>
{"name": "get_weather_forecast", "arguments": {"location": "San Francisco"}}
</tool_call>
<tool_call>
{"name": "get_stock_price", "arguments": {"symbol": "TSLA"}}
</tool_call>
<|im_end|>


In [40]:
import re
import json

def parse_tool_calls(response_text):
    tool_calls = []
    pattern = r'<tool_call>\s*(.*?)\s*</tool_call>'
    matches = re.findall(pattern, response_text)
    
    for match in matches:
        try:
            tool_call = json.loads(match)
            tool_calls.append({
                "type": "function",
                "function": {
                    "name": tool_call["name"],
                    "arguments": json.dumps(tool_call["arguments"])
                }
            })
        except json.JSONDecodeError:
            print(f"Failed to parse tool call: {match}")
    
    return tool_calls

In [41]:
tool_calls = parse_tool_calls(response)
print(tool_calls)

[{'type': 'function', 'function': {'name': 'get_weather_forecast', 'arguments': '{"location": "San Francisco"}'}}, {'type': 'function', 'function': {'name': 'get_stock_price', 'arguments': '{"symbol": "TSLA"}'}}]


In [45]:
messages.append({"role": "assistant", "tool_calls": tool_calls})

In [46]:
print(json.dumps(messages, indent=2))


[
  {
    "role": "user",
    "content": "Get the current weather forecast for San Francisco and get stock price of Tesla"
  },
  {
    "role": "assistant",
    "tool_calls": [
      {
        "type": "function",
        "function": {
          "name": "get_weather_forecast",
          "arguments": "{\"location\": \"San Francisco\"}"
        }
      },
      {
        "type": "function",
        "function": {
          "name": "get_stock_price",
          "arguments": "{\"symbol\": \"TSLA\"}"
        }
      }
    ]
  }
]


In [47]:

prompt = tokenizer.apply_chat_template(
    messages,
    tools=tools,
    tokenize=False
)
print(prompt)

<|begin_of_text|>You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools> {"type": "function", "function": {"name": "get_weather_forecast", "description": "get_weather_forecast(location: str) -> dict[str, str] - Retrieves a simple weather forecast for a given location.

    Args:
        location(str): The name of the location to get the weather forecast for.
    Returns:
        dict[str, str]: A dictionary containing the location, forecast, and temperature.
        If successful, the keys are 'location', 'forecast', and 'temperature'.
        If unsuccessful, the key is 'error' with a corresponding error message.", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The name of the location to get the weather forecast for

In [48]:
 for tool_call in tool_calls:
            function_call = tool_call["function"]
            name = function_call["name"]
            arguments = json.loads(function_call["arguments"])
            for function in tools:
                if function.__name__ == name:
                    print(f"Invoking tool call: {name}")
                    result = function(**arguments)
                    result_content = {
                        "name": name,
                        "content": result
                    }
                    messages.append(
                        {
                            "role": "tool",
                            "content": json.dumps(result_content),
                            "name": name
                        }
                    )

Invoking tool call: get_weather_forecast


Invoking tool call: get_stock_price


In [49]:
print(json.dumps(messages, indent=2))

[
  {
    "role": "user",
    "content": "Get the current weather forecast for San Francisco and get stock price of Tesla"
  },
  {
    "role": "assistant",
    "tool_calls": [
      {
        "type": "function",
        "function": {
          "name": "get_weather_forecast",
          "arguments": "{\"location\": \"San Francisco\"}"
        }
      },
      {
        "type": "function",
        "function": {
          "name": "get_stock_price",
          "arguments": "{\"symbol\": \"TSLA\"}"
        }
      }
    ]
  },
  {
    "role": "tool",
    "content": "{\"name\": \"get_weather_forecast\", \"content\": {\"location\": \"San Francisco\", \"forecast\": \"Clear \", \"temperature\": \"+55\\u00b0F\"}}",
    "name": "get_weather_forecast"
  },
  {
    "role": "tool",
    "content": "{\"name\": \"get_stock_price\", \"content\": \"$197.4900\"}",
    "name": "get_stock_price"
  }
]


In [50]:

prompt = tokenizer.apply_chat_template(
    messages,
    tools=tools,
    tokenize=False
)
print(prompt)

<|begin_of_text|>You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools> {"type": "function", "function": {"name": "get_weather_forecast", "description": "get_weather_forecast(location: str) -> dict[str, str] - Retrieves a simple weather forecast for a given location.

    Args:
        location(str): The name of the location to get the weather forecast for.
    Returns:
        dict[str, str]: A dictionary containing the location, forecast, and temperature.
        If successful, the keys are 'location', 'forecast', and 'temperature'.
        If unsuccessful, the key is 'error' with a corresponding error message.", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The name of the location to get the weather forecast for

In [51]:
def recursive_tool_calling(messages, tokenizer, model, tools):
    while True:
        assistant_message = run_hermes_tool_inference(messages, tokenizer, model, tools)
        
        tool_calls = parse_tool_calls(assistant_message)
        
        if tool_calls:
            messages.append({"role": "assistant", "tool_calls": tool_calls})
        else:
            messages.append({"role": "assistant", "content": assistant_message.strip("<|im_end|>")})
        
        print(f"Assistant Message: {assistant_message}")

        if not tool_calls:
            break

        for tool_call in tool_calls:
            function_call = tool_call["function"]
            name = function_call["name"]
            arguments = json.loads(function_call["arguments"])
            for function in tools:
                if function.__name__ == name:
                    print(f"Invoking tool call: {name}")
                    result = function(**arguments)
                    result_content = {
                        "name": name,
                        "content": result
                    }
                    messages.append(
                        {
                            "role": "tool",
                            "content": json.dumps(result_content),
                            "name": name
                        }
                    )
                    print(f"Tool Call Result: {result_content}")
                    break

    return messages

In [52]:
messages = recursive_tool_calling(messages, tokenizer, model, tools)


Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


Assistant Message: The current weather forecast for San Francisco is "Clear" and the temperature is +55°F. The stock price for Tesla (TSLA) is $197.49.<|im_end|>
